In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)
            
date = 0 #当日は0、前日は1・・・
date2 = "08/19"
pmc = "08024003" # 店舗ID
SPREADSHEET_KEY = '106CiDKzBpy0NkRU8rcsJxHX8f4i19V7oJDU7ul0NQyc' # スプレッドシートのID
pstart = 1 #1 パチンコの1台目
sstart = 701 #701 スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+789@gmail.com"
user_pw = "paruparu23"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
chrome_service = ChromeService(executable_path='C:\\Users\\garden026\\WD\\114.0.5735.90\\chromedriver.exe')

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "111":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "111":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})

['08/19', '025553', '1', '1282.0', '-1203.0', '0', '0']
['08/19', '025553', '2', '1795.0', '-1504.0', '0', '0']
['08/19', '025834', '3', '18205.0', '20464.0', '21', '5']
['08/19', '025834', '5', '2564.0', '9330.0', '7', '1']
['08/19', '025834', '6', '13333.0', '-2106.0', '5', '3']
['08/19', '025834', '7', '12051.0', '-3911.0', '3', '2']
['08/19', '025834', '8', '17949.0', '-2406.0', '7', '3']
['08/19', '025834', '10', '11026.0', '-9027.0', '0', '0']
['08/19', '025834', '11', '6410.0', '-1805.0', '2', '1']
['08/19', '025834', '12', '23333.0', '12038.0', '18', '5']
['08/19', '025834', '13', '2308.0', '-2106.0', '0', '0']
['08/19', '025834', '15', '1795.0', '-1504.0', '0', '0']
['08/19', '025834', '16', '6410.0', '-4814.0', '0', '0']
['08/19', '025834', '17', '7179.0', '603.0', '4', '1']
['08/19', '025834', '18', '25897.0', '-10231.0', '6', '3']
['08/19', '025892', '20', '26410.0', '6621.0', '40', '11']
['08/19', '025892', '21', '3590.0', '-902.0', '4', '2']
['08/19', '025780', '22', '205

['08/19', '025677', '226', '12564.0', '-6018.0', '5', '2']
['08/19', '025677', '227', '18205.0', '-7522.0', '6', '5']
['08/19', '025677', '228', '8205.0', '7223.0', '11', '1']
['08/19', '025677', '230', '27692.0', '-10832.0', '9', '4']
['08/19', '025867', '231', '2051.0', '-1805.0', '0', '0']
['08/19', '025867', '232', '6410.0', '5719.0', '8', '3']
['08/19', '025670', '233', '8205.0', '603.0', '11', '2']
['08/19', '025670', '235', '2564.0', '-1203.0', '1', '1']
['08/19', '025670', '236', '10513.0', '-902.0', '14', '2']
['08/19', '025606', '237', '5385.0', '-3911.0', '1', '1']
['08/19', '025606', '238', '10513.0', '6019.0', '10', '2']
['08/19', '025606', '250', '15641.0', '-12036.0', '0', '0']
['08/19', '025665', '251', '17949.0', '12941.0', '19', '3']
['08/19', '025665', '252', '24103.0', '12640.0', '22', '3']
['08/19', '025665', '253', '9487.0', '-7522.0', '1', '1']
['08/19', '025665', '255', '7949.0', '-6018.0', '1', '1']
['08/19', '025665', '256', '21538.0', '-4212.0', '14', '3']
['

['08/19', '025579', '551', '8462.0', '1506.0', '8', '1']
['08/19', '025317', '552', '15385.0', '-1203.0', '14', '6']
['08/19', '025436', '553', '7179.0', '603.0', '6', '2']
['08/19', '025436', '555', '6667.0', '-5416.0', '0', '0']
['08/19', '025436', '556', '20513.0', '1205.0', '16', '6']
['08/19', '025869', '557', '7692.0', '3612.0', '10', '1']
['08/19', '025869', '558', '17692.0', '-7823.0', '7', '1']
['08/19', '025869', '560', '3077.0', '-3008.0', '0', '0']
['08/19', '025869', '561', '11282.0', '-9027.0', '1', '1']
['08/19', '025737', '562', '12821.0', '12941.0', '46', '3']
['08/19', '025737', '563', '7179.0', '3010.0', '13', '2']
['08/19', '025737', '565', '11538.0', '-8124.0', '2', '1']
['08/19', '025472', '566', '16410.0', '-3309.0', '20', '10']
['08/19', '025472', '567', '15641.0', '-8726.0', '6', '3']
['08/19', '025509', '568', '3590.0', '603.0', '4', '1']
['08/19', '025509', '570', '15385.0', '-5717.0', '10', '6']
['08/19', '025455', '571', '18205.0', '-1504.0', '19', '5']
['0

['08/19', '120141', '785', '414', '-139.0', '1', '1', '--']
['08/19', '120141', '786', '5034', '278.0', '21', '16', '--']
['08/19', '120141', '787', '5551', '1806.0', '30', '18', '--']
['08/19', '120141', '788', '3533', '-926.0', '11', '6', '--']
['08/19', '120010', '800', '5727', '93.0', '22', '15', '--']
['08/19', '120010', '801', '5765', '-278.0', '19', '13', '--']
['08/19', '120010', '802', '2183', '-1019.0', '5', '5', '--']
['08/19', '120010', '803', '1864', '-833.0', '5', '1', '--']
['08/19', '120010', '805', '3766', '2917.0', '27', '5', '--']
['08/19', '120010', '806', '5600', '741.0', '22', '18', '--']
['08/19', '120010', '807', '4548', '787.0', '21', '13', '--']
['08/19', '120010', '808', '1759', '139.0', '7', '3', '--']
['08/19', '120010', '810', '3719', '1250.0', '20', '6', '--']
['08/19', '120010', '811', '3106', '-2037.0', '7', '4', '--']
['08/19', '119916', '812', '4933', '1435.0', '20', '20', '--']
['08/19', '119916', '813', '698', '-231.0', '2', '2', '--']
['08/19', '11

['08/19', '120145', '1101', '3236', '3704.0', '0', '4', '51']
['08/19', '120145', '1102', '6363', '-463.0', '0', '7', '56']
['08/19', '120145', '1103', '1481', '-1343.0', '0', '2', '6']
['08/19', '120145', '1105', '2057', '-1528.0', '0', '5', '12']
['08/19', '120145', '1106', '1896', '-972.0', '0', '4', '13']
['08/19', '120145', '1107', '4910', '-2037.0', '0', '8', '33']
['08/19', '119945', '1108', '4054', '787.0', '13', '8', '--']
['08/19', '120015', '1110', '4', '0', '0', '0', '--']
['08/19', '119929', '1111', '936', '1852.0', '0', '1', '8']
['08/19', '119926', '1112', '1301', '-370.0', '0', '1', '12']
['08/19', '119883', '1113', '1822', '-139.0', '8', '0', '2']
['08/19', '120094', '1115', '239', '-139.0', '0', '0', '1']
['08/19', '119865', '1116', '569', '-93.0', '0', '1', '4']
['08/19', '120072', '1117', '2767', '2083.0', '11', '3', '14']
['08/19', '119879', '1118', '135', '-139.0', '0', '0', '--']
['08/19', '120148', '1120', '3805', '-4028.0', '0', '7', '10']
['08/19', '120148', '

{'spreadsheetId': '106CiDKzBpy0NkRU8rcsJxHX8f4i19V7oJDU7ul0NQyc',
 'updates': {'spreadsheetId': '106CiDKzBpy0NkRU8rcsJxHX8f4i19V7oJDU7ul0NQyc',
  'updatedRange': 'S20RAW!F56018:M56259',
  'updatedRows': 242,
  'updatedColumns': 8,
  'updatedCells': 1936}}